Lexi Campanilla

First Arcpy Lab

In [1]:
import sys
sys.path.append('D:\\Desktop10.5\\bin')
sys.path.append('D:\\Desktop10.5\\arcpy')
sys.path.append('D:\\Desktop10.5\\ArcToolbox\\Scripts')
import arcpy

In [2]:
from arcpy import env
env.workspace = "C:\\Users\\Alexandra\\Documents\\GEOG458\\firstArcpyLab\\"

In [3]:
shapefile="C:\\Users\\Alexandra\\Documents\\GEOG458\\firstArcpyLab\\saep_bg10.shp"
WAfips="C:\\Users\\Alexandra\\Documents\\GEOG458\\firstArcpyLab\\WashingtonFIPS.dbf"

Reinstalled osgeo4w64

In [4]:
from subprocess import call
import os
os.environ["GDAL_DATA"]="C:\\Users\\Alexandra\\Downloads\\OSGeo4W64-20180216T050230Z-001\\OSGeo4W64\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"]="C:\\Users\\Alexandra\\Downloads\\OSGeo4W64-20180216T050230Z-001\\OSGeo4W64\\bin\\gdalplugins"
os.environ["PROJ_LIB"]="C:\\Users\\Alexandra\\Downloads\\OSGeo4W64-20180216T050230Z-001\\OSGeo4W64\\share\\proj"
os.environ["PATH"]="C:\\Users\\Alexandra\\Downloads\\OSGeo4W64-20180216T050230Z-001\\OSGeo4W64\\bin;"+os.environ["PATH"]+";C:\\Users\\Alexandra\\Downloads\\OSGeo4W64-20180216T050230Z-001\\OSGeo4W64\\apps\\msys\\bin;C:\\Users\\Alexandra\\Downloads\\OSGeo4W64-20180216T050230Z-001\\OSGeo4W64\\apps\\Python27\\Scripts"

adds COUNTYname field in the shapefile

loop shapefile to put counties with same FIPS code into COUNTYname

In [5]:
arcpy.management.AddField(shapefile,"COUNTYname","TEXT")
                              
mylist=[]
cursor=arcpy.da.SearchCursor(WAfips,["CountyName","FIPSCounty"]) #searches WAfips for CountyName and FIPScounty
for row in cursor:
    
    cursor2=arcpy.da.UpdateCursor(shapefile,"COUNTYname",'"COUNTYFP10" = ' + "'%s'"%row[1]) #makes it a string
    mylist.append(row[0])
    for row2 in cursor2:
        row2[0] = row[0]
        cursor2.updateRow(row2)
del cursor
del cursor2          

verification

In [6]:
fieldlist=arcpy.ListFields(shapefile)
for curr_field in fieldlist:
    print curr_field.name

FID
Shape
STATEFP10
COUNTYFP10
TRACTCE10
BLKGRPCE10
GEOID10
NAMELSAD10
MTFCC10
FUNCSTAT10
INTPTLON10
INTPTLAT10
ALANDM
AWATERM
ALANDMI
AWATERMI
Version
POP2000
POP2001
POP2002
POP2003
POP2004
POP2005
POP2006
POP2007
POP2008
POP2009
POP2010
POP2011
POP2012
POP2013
POP2014
POP2015
POP2016
POP2017
CPOP00_10
PCPOP00_10
CPOP10_17
PCPOP10_17
HHP2000
HHP2001
HHP2002
HHP2003
HHP2004
HHP2005
HHP2006
HHP2007
HHP2008
HHP2009
HHP2010
HHP2011
HHP2012
HHP2013
HHP2014
HHP2015
HHP2016
HHP2017
CHHP00_10
PCHHP00_10
CHHP10_17
PCHHP10_17
GQ2000
GQ2001
GQ2002
GQ2003
GQ2004
GQ2005
GQ2006
GQ2007
GQ2008
GQ2009
GQ2010
GQ2011
GQ2012
GQ2013
GQ2014
GQ2015
GQ2016
GQ2017
CGQ00_10
PCGQ00_10
CGQ10_17
PCGQ10_17
HU2000
HU2001
HU2002
HU2003
HU2004
HU2005
HU2006
HU2007
HU2008
HU2009
HU2010
HU2011
HU2012
HU2013
HU2014
HU2015
HU2016
HU2017
CHU00_10
PCHU00_10
CHU10_17
PCHU10_17
OHU2000
OHU2001
OHU2002
OHU2003
OHU2004
OHU2005
OHU2006
OHU2007
OHU2008
OHU2009
OHU2010
OHU2011
OHU2012
OHU2013
OHU2014
OHU2015
OHU2016
OHU2017
COHU

makes files for each county

In [7]:
for WAcounty in mylist:
    outLocation="C:\\Users\\Alexandra\\Documents\\GEOG458\\firstArcpyLab\\"
    arcpy.FeatureClassToFeatureClass_conversion(shapefile, outLocation,str(WAcounty),'"COUNTYname"='+"'%s'"%WAcounty)

as seen below

In [8]:
print mylist

[u'Adams', u'Asotin', u'Benton', u'Chelan', u'Clallam', u'Clark', u'Columbia', u'Cowlitz', u'Douglas', u'Ferry', u'Franklin', u'Garfield', u'Grant', u'Grays Harbor', u'Island', u'Jefferson', u'King', u'Kitsap', u'Kittitas', u'Klickitat', u'Lewis', u'Lincoln', u'Mason', u'Okanogan', u'Pacific', u'Pend Oreille', u'Pierce', u'San Juan', u'Skagit', u'Skamania', u'Snohomish', u'Spokane', u'Stevens', u'Thurston', u'Wahkiakum', u'Walla Walla', u'Whatcom', u'Whitman', u'Yakima']


produces geojson files

In [9]:
for WAcounty in mylist:
    call(['C:\\Users\\Alexandra\\Downloads\\OSGeo4W64-20180216T050230Z-001\\OSGeo4W64\\bin\\ogr2ogr',
      '-f','GeoJSON','-t_srs','WGS84',
      '-s_srs','EPSG:2927',
      'C:\\Users\\Alexandra\\Documents\\GEOG458\\firstArcpyLab\\geojsoncounties\\'+str(WAcounty)+'.geojson',
      outLocation+str(WAcounty)+'.shp'])

adds population of the counties

In [16]:
top10=[]
for WAcounty in mylist:
    cursor=arcpy.da.SearchCursor(shapefile,["POP2017"],'"COUNTYname"='+"'%s'"%WAcounty)
    population=0
    for row in cursor:
        population+=row[0]
    top10.append(population)
del cursor
print top10    

[19870.0, 22290.0, 193499.99999999997, 76829.99999999999, 74240.00000000001, 470999.9999999994, 4100.0, 105900.00000000001, 41420.0, 7740.0, 90330.00000000003, 2200.0, 95630.0, 72970.0, 82790.0, 31360.0, 2153700.0, 264300.00000000006, 44730.0, 21660.0, 77440.00000000003, 10700.0, 63190.00000000001, 42110.00000000001, 21249.999999999996, 13370.0, 859400.0000000003, 16510.0, 124100.00000000001, 11690.0, 789400.0000000005, 499800.0, 44510.0, 276899.9999999999, 4030.0, 61399.999999999985, 216300.00000000003, 48639.99999999999, 253000.0]


sorts top 10 populations (highest)

In [17]:
top10.sort(reverse=True)
for i in range(0,10):
    print top10[i]

2153700.0
859400.0
789400.0
499800.0
471000.0
276900.0
264300.0
253000.0
216300.0
193500.0
